In [24]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.15)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.15)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.15)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(16 * 3 * 3, 10)
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(-1, 16 * 3 * 3)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [26]:
#!pip install torchinfo
from torchinfo import summary
use_cuda = torch.cuda.is_available()
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if use_cuda else "cpu")
model = Net().to(device)
# Create a dummy input tensor on the correct device
summary(model, input_size=(1, 1, 28, 28), device=device)

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [1, 10]                   --
├─Sequential: 1-1                        [1, 16, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 8, 28, 28]            80
│    └─ReLU: 2-2                         [1, 8, 28, 28]            --
│    └─BatchNorm2d: 2-3                  [1, 8, 28, 28]            16
│    └─Conv2d: 2-4                       [1, 16, 28, 28]           1,168
│    └─ReLU: 2-5                         [1, 16, 28, 28]           --
│    └─BatchNorm2d: 2-6                  [1, 16, 28, 28]           32
│    └─MaxPool2d: 2-7                    [1, 16, 14, 14]           --
│    └─Dropout: 2-8                      [1, 16, 14, 14]           --
├─Sequential: 1-2                        [1, 32, 7, 7]             --
│    └─Conv2d: 2-9                       [1, 16, 14, 14]           2,320
│    └─ReLU: 2-10                        [1, 16, 14, 14]           --
│    └─Ba

In [27]:


torch.manual_seed(1456)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if device.type in ["cuda", "mps"] else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# print number of samples in train and test dataset
print(f"Number of samples in train dataset: {len(train_loader.dataset)}")
print(f"Number of samples in test dataset: {len(test_loader.dataset)}")


Number of samples in train dataset: 60000
Number of samples in test dataset: 10000


In [28]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

loss=0.12121749669313431 batch_id=468: 100%|██████████| 469/469 [00:04<00:00, 102.92it/s] 



Test set: Average loss: 0.0460, Accuracy: 9854/10000 (98.54%)



loss=0.09349746257066727 batch_id=468: 100%|██████████| 469/469 [00:04<00:00, 101.31it/s] 



Test set: Average loss: 0.0296, Accuracy: 9906/10000 (99.06%)



...


Test set: Average loss: 0.0180, Accuracy: 9942/10000 (99.42%)



loss=0.0032117466907948256 batch_id=468: 100%|██████████| 469/469 [00:04<00:00, 109.01it/s]



Test set: Average loss: 0.0186, Accuracy: 9940/10000 (99.40%)



loss=0.016841059550642967 batch_id=468: 100%|██████████| 469/469 [00:04<00:00, 108.69it/s] 



Test set: Average loss: 0.0218, Accuracy: 9922/10000 (99.22%)

